In [70]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.preprocessing import sequence

from TransformerComplet import *

from tensorflow.keras.layers import Dense, Dropout, Embedding, Input, Concatenate, MultiHeadAttention


In [71]:


# Chargement du jeu de données
data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")

In [73]:
# Convertir toutes les colonnes catégorielles en chaînes de caractères

#all_data = data.astype(str)


all_data =data
print(all_data.dtypes)


Domain                   int64
DNSRecordType           object
MXDnsResponse             bool
TXTDnsResponse            bool
HasSPFInfo                bool
HasDkimInfo               bool
HasDmarcInfo              bool
Ip                       int64
DomainInAlexaDB           bool
CommonPorts               bool
CountryCode             object
RegisteredCountry       object
CreationDate             int64
LastUpdateDate           int64
ASN                      int64
HttpResponseCode         int64
RegisteredOrg           object
SubdomainNumber          int64
Entropy                  int64
EntropyOfSubDomains      int64
StrangeCharacters        int64
TLD                     object
IpReputation              bool
DomainReputation          bool
ConsoantRatio          float64
NumericRatio           float64
SpecialCharRatio       float64
VowelRatio             float64
ConsoantSequence         int64
VowelSequence            int64
NumericSequence          int64
SpecialCharSequence      int64
DomainLe

In [74]:
print(all_data['Class'].unique())


[1 0]


In [41]:
print(all_data.info())

# print(all_data['Class'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Domain               90000 non-null  object
 1   DNSRecordType        90000 non-null  object
 2   MXDnsResponse        90000 non-null  object
 3   TXTDnsResponse       90000 non-null  object
 4   HasSPFInfo           90000 non-null  object
 5   HasDkimInfo          90000 non-null  object
 6   HasDmarcInfo         90000 non-null  object
 7   Ip                   90000 non-null  object
 8   DomainInAlexaDB      90000 non-null  object
 9   CommonPorts          90000 non-null  object
 10  CountryCode          90000 non-null  object
 11  RegisteredCountry    90000 non-null  object
 12  CreationDate         90000 non-null  object
 13  LastUpdateDate       90000 non-null  object
 14  ASN                  90000 non-null  object
 15  HttpResponseCode     90000 non-null  object
 16  Regi

In [98]:
data.isnull().sum().sort_values(ascending=False)

#si c'est 0 partout c'est que c'est ok si c'est les NAN c'est qu'il y'a les valeurs manquantes et il fait va falloir dont leurs travailler pour completer ou suspendre cela



RegisteredCountry      77774
RegisteredOrg          35391
CountryCode            29052
TLD                      170
Domain                     0
EntropyOfSubDomains        0
IpReputation               0
DomainReputation           0
ConsoantRatio              0
NumericRatio               0
SpecialCharRatio           0
VowelRatio                 0
ConsoantSequence           0
VowelSequence              0
NumericSequence            0
SpecialCharSequence        0
DomainLength               0
StrangeCharacters          0
SubdomainNumber            0
Entropy                    0
DNSRecordType              0
HttpResponseCode           0
ASN                        0
LastUpdateDate             0
CreationDate               0
CommonPorts                0
DomainInAlexaDB            0
Ip                         0
HasDmarcInfo               0
HasDkimInfo                0
HasSPFInfo                 0
TXTDnsResponse             0
MXDnsResponse              0
Class                      0
dtype: int64

In [42]:
# Sélectionner la colonne 'Class' avant l'encodage
y = all_data['Class']

In [43]:
print(y)

0        1
1        1
2        1
3        1
4        1
        ..
89995    0
89996    0
89997    0
89998    0
89999    0
Name: Class, Length: 90000, dtype: object


In [44]:
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Domain               90000 non-null  object
 1   DNSRecordType        90000 non-null  object
 2   MXDnsResponse        90000 non-null  object
 3   TXTDnsResponse       90000 non-null  object
 4   HasSPFInfo           90000 non-null  object
 5   HasDkimInfo          90000 non-null  object
 6   HasDmarcInfo         90000 non-null  object
 7   Ip                   90000 non-null  object
 8   DomainInAlexaDB      90000 non-null  object
 9   CommonPorts          90000 non-null  object
 10  CountryCode          90000 non-null  object
 11  RegisteredCountry    90000 non-null  object
 12  CreationDate         90000 non-null  object
 13  LastUpdateDate       90000 non-null  object
 14  ASN                  90000 non-null  object
 15  HttpResponseCode     90000 non-null  object
 16  Regi

In [45]:
print(all_data.shape)


(90000, 34)


In [ ]:
################################ Ne pas executer c'esr juste un exemple ##############


############## Tresimportant pour selectionner les attribut numeriques ###### il faut d'abord affihcer les types des donnees ##########

# Supposons que X_numerical_part1 et X_numerical_part2 sont deux parties de votre DataFrame avec des colonnes numériques.
X_numerical_part1 = all_data.select_dtypes(include=['int64', 'float64'])
X_numerical_part2 = all_data.select_dtypes(include=['int64', 'float64'])

# Concaténer les parties avec les colonnes numériques
X_numerical = pd.concat([X_numerical_part1, X_numerical_part2], axis=1)


In [81]:
# Séparer les caractéristiques numériques et catégorielles

#X_numerical = all_data.select_dtypes(include='number') ########### jai du mal a comprendre comment il fonctionne lui

X_numerical = all_data.select_dtypes(include=['int64', 'float64'])

X_categorical = all_data.select_dtypes(exclude='number')

In [80]:
print(all_data.shape)
print(X_numerical.shape)
print(X_categorical.shape)

(90000, 34)
(90000, 20)
(90000, 14)


In [82]:
from sklearn.preprocessing import LabelEncoder

# Encodage des variables catégorielles
categorical_features = X_categorical.columns
label_encoder = LabelEncoder()
X_categorical_encoded = X_categorical.apply(label_encoder.fit_transform)

# Utiliser l'encodage ordinal pour transformer les données catégorielles en valeurs numériques

#ordinal_encoder = OrdinalEncoder()
#X_categorical_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_categorical), columns=X_categorical.columns)


In [83]:
print(X_numerical)

       Domain     Ip  CreationDate  LastUpdateDate    ASN  HttpResponseCode   
0        4455  16984             0               0     -1                 0  \
1        4456  16984             0               0     -1                 0   
2        4457  16984             0               0     -1                 0   
3        4458  16984             0               0     -1                 0   
4        4459  16984             0               0     -1                 0   
...       ...    ...           ...             ...    ...               ...   
89995   75661  16984             0               0     -1                 0   
89996   54843  16984             0               0     -1                 0   
89997   71235  16984             0               0     -1                 0   
89998   82994   7723             4               4  46606                 4   
89999   64601  16984             0               0     -1                 0   

       SubdomainNumber  Entropy  EntropyOfSubDomain

In [84]:
############## j'affiche la variance des differents features categoricielles #######

X_categorical_encoded.var()


DNSRecordType        3.494257e-01
MXDnsResponse        1.877407e-01
TXTDnsResponse       2.499788e-01
HasSPFInfo           2.499279e-01
HasDkimInfo          2.443874e-04
HasDmarcInfo         1.950421e-02
DomainInAlexaDB      2.305388e-02
CommonPorts          1.143318e-01
CountryCode          5.573118e+02
RegisteredCountry    6.568051e+02
RegisteredOrg        1.710584e+06
TLD                  3.224950e+04
IpReputation         4.310690e-02
DomainReputation     1.198573e-03
dtype: float64

In [85]:
print(X_categorical_encoded.shape)


(90000, 14)


In [86]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OrdinalEncoder

# Mettez à jour la sélection des colonnes originales

# Réduire la dimension  [[on va supprimer les features categorique qui ont une variance inferieur au seuil]] avec VarianceThreshold varie le Threshold # Le selector va eleminer certain attribut categoricel  sonc la variance est inferieur a celle fixe
selector = VarianceThreshold(threshold=0.2)
X_categorical_reduced = selector.fit_transform(X_categorical_encoded)

# Mettez à jour la sélection des colonnes originales
original_categorical_columns = X_categorical.columns[selector.get_support()]


In [87]:
print( original_categorical_columns)
print( original_categorical_columns.shape) ### variable restante apres suppression

Index(['DNSRecordType', 'TXTDnsResponse', 'HasSPFInfo', 'CountryCode',
       'RegisteredCountry', 'RegisteredOrg', 'TLD'],
      dtype='object')
(7,)


In [88]:
# Créer un DataFrame avec les colonnes originales
X_categorical_reduced_df = pd.DataFrame(X_categorical_reduced, columns=original_categorical_columns)


In [89]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# 2. Vérification des colonnes vides
if X_numerical.isnull().any().any():
    # Imputer les valeurs manquantes pour les caractéristiques numériques
    numerical_imputer = SimpleImputer(strategy='mean')
    X_numerical_imputed = pd.DataFrame(numerical_imputer.fit_transform(X_numerical), columns=X_numerical.columns)

    #prepocessing des features numeriques soit  avec le  LabelEncoder soit le MinMaxScaler()

    # Création d'un scaler
    scaler = MinMaxScaler()

    # Ajustement du scaler aux données
    scaler.fit(X_numerical_imputed)

    # Transformation des fonctionnalités numériques
    scaled_numeric_features = scaler.transform(X_numerical_imputed)


    print(scaled_numeric_features)

    # Apres transformation Création d' un DataFrame à partir des valeurs transformées

    scaled_df = pd.DataFrame(scaled_numeric_features, columns=X_numerical_imputed.columns)

    # Afficher le DataFrame avec les valeurs transformées
    print("Après transformation")
    print(scaled_df)

    total_size = scaled_df.shape
    print("Taille totale des caractéristiques numériques après transformations :", total_size)
else:
    print("Pas de valeurs manquantes dans les caractéristiques numériques. Aucune imputation nécessaire.")


Pas de valeurs manquantes dans les caractéristiques numériques. Aucune imputation nécessaire.


In [90]:
#Concaténer les caractéristiques numériques et catégorielles
X_combined = pd.concat([X_numerical, X_categorical_reduced_df], axis=1)

In [91]:
num_features = X_combined.shape[1]
print("Nombre total de caractéristiques numeriques et categoriciel:", num_features)


Nombre total de caractéristiques numeriques et categoriciel: 27


In [92]:
# Imprimer les noms des colonnes pour déboguer
print(X_combined.columns)


Index(['Domain', 'Ip', 'CreationDate', 'LastUpdateDate', 'ASN',
       'HttpResponseCode', 'SubdomainNumber', 'Entropy', 'EntropyOfSubDomains',
       'StrangeCharacters', 'ConsoantRatio', 'NumericRatio',
       'SpecialCharRatio', 'VowelRatio', 'ConsoantSequence', 'VowelSequence',
       'NumericSequence', 'SpecialCharSequence', 'DomainLength', 'Class',
       'DNSRecordType', 'TXTDnsResponse', 'HasSPFInfo', 'CountryCode',
       'RegisteredCountry', 'RegisteredOrg', 'TLD'],
      dtype='object')


In [93]:
from sklearn.ensemble import ExtraTreesClassifier

def feature_importance(X_combined,Y,num_features):
    model = ExtraTreesClassifier()
    model.fit(X_combined,Y)

    feature_importances = pd.Series(model.feature_importances_ ,index=X_combined.columns )
    return feature_importances.nlargest(num_features).axes[0].values


In [94]:
#La Class ne doit pas faire partir des features important car c'est la cible

# Imprimer les noms des colonnes pour déboguer
print(X_combined.columns)

# Supprimer les colonnes de classe s'il y en a
# class_columns_to_drop = ['Class_1', 'Class_2', 'Class_3']

class_columns_to_drop = ['Class']

existing_class_columns = set(class_columns_to_drop).intersection(X_combined.columns)

if existing_class_columns:
    X_combined = X_combined.drop(existing_class_columns, axis=1)

# Imprimer à nouveau les noms des colonnes pour vérifier
print(X_combined.columns)


Index(['Domain', 'Ip', 'CreationDate', 'LastUpdateDate', 'ASN',
       'HttpResponseCode', 'SubdomainNumber', 'Entropy', 'EntropyOfSubDomains',
       'StrangeCharacters', 'ConsoantRatio', 'NumericRatio',
       'SpecialCharRatio', 'VowelRatio', 'ConsoantSequence', 'VowelSequence',
       'NumericSequence', 'SpecialCharSequence', 'DomainLength', 'Class',
       'DNSRecordType', 'TXTDnsResponse', 'HasSPFInfo', 'CountryCode',
       'RegisteredCountry', 'RegisteredOrg', 'TLD'],
      dtype='object')
Index(['Domain', 'Ip', 'CreationDate', 'LastUpdateDate', 'ASN',
       'HttpResponseCode', 'SubdomainNumber', 'Entropy', 'EntropyOfSubDomains',
       'StrangeCharacters', 'ConsoantRatio', 'NumericRatio',
       'SpecialCharRatio', 'VowelRatio', 'ConsoantSequence', 'VowelSequence',
       'NumericSequence', 'SpecialCharSequence', 'DomainLength',
       'DNSRecordType', 'TXTDnsResponse', 'HasSPFInfo', 'CountryCode',
       'RegisteredCountry', 'RegisteredOrg', 'TLD'],
      dtype='object')


In [95]:
print(X_combined.shape)


(90000, 26)


In [96]:
# Appel de la fonction et impression des features importants

#donc dans notre fonction il faut passer un nombre de caracteristiques inferieur ou egale a 13 avec la Methode ExtrareesClassifier sa ne derange pas mem si tu donne une valeur superieur comparer au Univariate (SelectKBest,chi2)

important_features = feature_importance(X_combined, y, num_features=22)
print("Features importants:")
print(important_features)

Features importants:
['DNSRecordType' 'Domain' 'NumericRatio' 'HasSPFInfo' 'ConsoantRatio'
 'TXTDnsResponse' 'DomainLength' 'VowelRatio' 'CreationDate'
 'StrangeCharacters' 'NumericSequence' 'LastUpdateDate' 'RegisteredOrg'
 'Ip' 'HttpResponseCode' 'CountryCode' 'TLD' 'Entropy' 'ConsoantSequence'
 'VowelSequence' 'RegisteredCountry' 'ASN']


In [97]:
print(important_features.shape)


(22,)
